In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import widgets

# Helper function to calculate similarity
def calculate_similarity(image1_path, image2_path):
    image1 = Image.open(image1_path).convert('L')
    image2 = Image.open(image2_path).convert('L')

    image1 = image1.resize((256, 256))
    image2 = image2.resize((256, 256))

    image1_np = np.array(image1)
    image2_np = np.array(image2)

    similarity_index, _ = ssim(image1_np, image2_np, full=True)
    return similarity_index


In [ ]:
# Widget to upload an image
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
display(upload_widget)

In [ ]:
def find_sole_images(root_dir):
    sole_images = []
    valid_extensions = {'.png', '.jpg', '.jpeg', '.jfif', '.bmp', '.gif', '.tiff'}
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_name, file_extension = os.path.splitext(file)
            if file_name.lower() == 'sole' and file_extension.lower() in valid_extensions:
                sole_images.append(os.path.join(subdir, file))
    return sole_images


In [ ]:
def compare_uploaded_image(upload_widget, root_dir):
    if upload_widget.value:
        uploaded_image = list(upload_widget.value.values())[0]['content']
        uploaded_image_path = '/mnt/data/uploaded_image.png'
        
        with open(uploaded_image_path, 'wb') as f:
            f.write(uploaded_image)
        
        sole_images = find_sole_images(root_dir)
        similarities = []
        
        for sole_image in sole_images:
            similarity = calculate_similarity(uploaded_image_path, sole_image)
            # Add the similarity score and the sole image path to the results  
            similarities.append((os.path.relpath(sole_image, root_dir), similarity))
        
        # Sort the results by similarity in descending order - the most likely
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Create a DataFrame to store the results
        df = pd.DataFrame(similarities, columns=['File Path', 'Similarity'])
        
        # Save the results to a CSV file
        csv_path = '/mnt/data/similarity_results.csv'
        df.to_csv(csv_path, index=False)
        
        # Display the results as a table
        display(HTML(df.to_html(index=False)))
        
        print(f'Results saved to {csv_path}')
    else:
        print("Please upload an image first.")


In [ ]:
root_dir = '/mnt/data/Shoe'  # Need to check this is the right file path 

compare_button = widgets.Button(description="Compare Images")
display(compare_button)

def on_compare_button_clicked(b):
    compare_uploaded_image(upload_widget, root_dir)

compare_button.on_click(on_compare_button_clicked)
